In [1]:
!pip install openai
!pip install bidict

In [ ]:
!pip install --upgrade accelerate transformers

In [2]:
from typing import List
from openai import OpenAI
import yaml
import json
import bidict
from random import shuffle
from typing import Tuple
from math import ceil
from random import sample
import numpy as np

with open('conf.yaml', 'r') as file:
    conf = yaml.safe_load(file)

client = OpenAI(api_key=conf['openai-key'])

In [ ]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM

In [3]:
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

artists = load_json("nlp_artists_filtered.json")
seeds = load_json("nlp_seeds_anonymized.json")
artist_ids = [k for k in artists.keys()]

In [39]:
# This is an interface with two implementations;
# GPTRecommender which implements everything by making API calls to ChatGPT's API
# InternalRecommender which implements everything by generating responses from an internal model
# - For this experiment, we will use one of Google's open-source Gemma models
class Recommender:

    def __init__(self, artists, prompt):
        # essentially json of {artist_id: artist_data (genres tags, wiki, etc)}
        self.artists = artists
        self.prompt = prompt

    def recommend(self, seed_ids: List[str], candidate_ids: List[str], use_genres=False) -> List[str]:
        raise NotImplementedError(f'{self.__class__} class is an interface and not intended for instantiation.')

    def parse_output(self, output, candidate_dict):
        start_index = output.rfind('<') + 1
        end_index = output.rfind('>')
        if start_index == -1 or end_index == -1:
            return None
        cleaned = output[start_index:end_index]
        try:
            iids = map(int, cleaned.split(','))
            return [candidate_dict.inverse[i] for i in iids if i in candidate_dict.inverse]
        except Exception as e:
            return None

    def construct_artist_string(self, _id, use_genres=False, use_wiki=False):
        if use_genres:
            return self.artists[_id]['name'] + f" ({', '.join(self.artists[_id]['genres'])})"
        if use_wiki:
            wiki_clipped = ' - ' + '. '.join(self.artists[_id]['wiki'].split('. ')[:5])
            return self.artists[_id]['name'] + wiki_clipped
        return self.artists[_id]['name']


class GPTRecommender(Recommender):

    def __init__(self, artists, prompt, client):
        super().__init__(artists, prompt)
        self.client = client

    def recommend(self, seed_ids: List[str], candidate_ids: List[str], use_genres=False, use_wiki=False) -> List[str]:
        #print('Seeds: ')
        print(seed_ids)
        #print('Candidates: ')
        print(candidate_ids)
        iids = list(range(len(candidate_ids)))
        candidate_dict = bidict.bidict({candidate_ids[i]: iids[i] for i in range(len(candidate_ids))})
        join_char = '\n' if not use_wiki else '\n\n'
        seed_names = join_char.join([self.construct_artist_string(_id, use_genres, use_wiki) for _id in seed_ids])
        candidate_names = join_char.join([self.construct_artist_string(_id, use_genres, use_wiki) for _id in candidate_ids])
        candidate_dict_text = '\n'.join([f"{candidate_dict[_id]}: {self.artists[_id]['name']}" for _id in candidate_ids])
        _prompt = self.prompt.format(seeds=seed_names, candidates=candidate_names, candidate_key=candidate_dict_text)
        #print(_prompt)
        response = self.client.responses.create(
            model="gpt-4o-mini",
            input=_prompt
        )
        text = response.output_text.replace('#', '')
        print("Response Text: ")
        print(text, end="\n\n")
        return self.parse_output(text, candidate_dict)


In [33]:
class GemmaWrapper:

    def __init__(self, model, tokenizer, system_prompt):
        self.model = model
        self.tokenizer = tokenizer
        self.system_prompt = system_prompt

    def get_messages(self, system_message, user_message):
        messages = [
            [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": system_message},]
                },
                {
                    "role": "user",
                    "content": [{"type": "text", "text": user_message},]
                },
            ],
        ]
        return messages

    def get_response(self, prompt):
        messages = self.get_messages(self.system_prompt, prompt)

        inputs = self.tokenizer.apply_chat_template(
            messages, add_generation_prompt=True, tokenize=True,
            return_dict=True, return_tensors="pt"
        ).to(self.model.device)

        input_len = inputs["input_ids"].shape[-1]

        generation = self.model.generate(**inputs, max_new_tokens=500, do_sample=False)
        generation = generation[0][input_len:]

        decoded = self.tokenizer.decode(generation, skip_special_tokens=True)
        print(decoded)
        return decoded


class InternalRecommender(Recommender):

    def __init__(self, artists, prompt, gemma_wrapper):
        super().__init__(artists, prompt)
        self.gw = gemma_wrapper

    def recommend(self, seed_ids: List[str], candidate_ids: List[str], use_genres=False, use_wiki=False) -> List[str]:
        #print('Seeds: ')
        print(seed_ids)
        #print('Candidates: ')
        print(candidate_ids)
        iids = list(range(len(candidate_ids)))
        join_char = '\n' if not use_wiki else '\n\n'
        candidate_dict = bidict.bidict({candidate_ids[i]: iids[i] for i in range(len(candidate_ids))})
        seed_names = join_char.join([self.construct_artist_string(_id, use_genres, use_wiki) for _id in seed_ids])
        candidate_names = join_char.join([self.artists[_id]['name'] + ('' if not use_genres else f" ({', '.join(self.artists[_id]['genres'])})") for _id in candidate_ids])
        candidate_dict_text = '\n'.join([f"{candidate_dict[_id]}: {self.artists[_id]['name']}" for _id in candidate_ids])
        _prompt = self.prompt.format(seeds=seed_names, candidates=candidate_names, candidate_key=candidate_dict_text)
        text = self.gw.get_response(_prompt)
        print("Response Text: ")
        print(text, end="\n\n")
        return self.parse_output(text, candidate_dict)

In [10]:
def calc_auc_score(rank_relevance):
    """
    usage : result = model.calc_auc_score([1,0,1,0,0,0,1,0,0,0,0,0])
    :param rank_relevance: list of 1s (relevant) and 0s (not relevant)
    :return: AUC score between 0 and 1. 0.5 is random. 1.0 is perfect (all relevant items at the top.)
    """
    num_true = sum(rank_relevance)
    num_false = len(rank_relevance) - num_true

    if num_true == 0 or num_false == 0:
        return -1

    tpr = 0
    total = 0
    for val in rank_relevance:
        if val:
            tpr += 1
        else:
            total += tpr

    auc = total / (num_true * num_false)
    return auc

In [30]:
class Evaluator:

    def __init__(self, recommender, artist_ids, seed_lists):
        self.recommender = recommender
        self.artist_ids = artist_ids
        self.seed_lists = seed_lists

    def do_trial(self, seed_ids, use_genres=False, use_wiki=False):
        split = ceil(len(seed_ids) * 0.5)
        shuffle(seed_ids)
        masked = seed_ids[split:]
        shuffle(masked)
        masked = masked[:30] if len(masked) >= 30 else masked
        unmasked = seed_ids[:split]

        potential_distractors = [_id for _id in self.artist_ids if _id not in seed_ids]
        distractors = sample(potential_distractors, len(masked))

        candidates = masked + distractors
        shuffle(candidates)
        results = self.recommender.recommend(unmasked, candidates, use_genres, use_wiki)
        if results is None:
            return -1

        relevances = [1 if res in masked else 0 for res in results]

        # ChatGPT doesn't always return the full list, so pad with a sampled list of 1s and 0s according to the appropriate proportions
        size_difference = len(candidates) - len(relevances)
        print('Difference', size_difference)
        if size_difference > 0:
            num_pos = len([x for x in relevances if x == 1])
            prop_pos = num_pos / len(relevances)
            pad_neg = ceil(size_difference * prop_pos)
            pad_pos = size_difference - pad_neg
            padding = [0 for i in range(pad_neg)] + [1 for i in range(pad_pos)]
            shuffle(padding)
            print('Padding:', padding)
            relevances += padding

        print(relevances)
        return calc_auc_score(relevances)

    def eval_model(self, use_genres=False, use_wiki=False):
        scores = []
        for seed_ids in self.seed_lists:
            score = self.do_trial(seed_ids, use_genres, use_wiki)
            print('Score:', score)
            if score != -1:
                scores.append(score)
        print(scores)
        return np.mean(scores)

In [7]:
ckpt = "google/gemma-3-4b-it"
gemma_model = Gemma3ForCausalLM.from_pretrained(
    ckpt,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_auth_token=conf['gemma-key'],
    low_cpu_mem_usage=True,
)

gemma_tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-3-4b-it",
    use_auth_token=conf['gemma-key'],
)

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4056: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [7]:
prompt_1_gpt = """You are an expert in music recommendation. Your specialty is in recommending local artists based on a user's specified, preferred artists.
You are presented with a client who frequently listens to the following artists:
Ariana Grande
Fleetwood Mac
BlackPink
Lady Gaga
Drake
The Weeknd

You are asked to use your expert knowledge of these artists and your specialiation in local music to recommend a set of 5 artists from Ithaca, New York.
"""
response = client.responses.create(
    model="gpt-4o-mini",
    input=prompt_1_gpt,
    tools=[{"type": "web_search_preview"}]
)
text = response.output_text
print(text)

Great choice of artists! Based on your client's preferences for pop, R&B, and indie styles, here are five local artists from Ithaca, New York, that they might enjoy:

1. **Joan M.** - A pop singer-songwriter with a dreamy sound reminiscent of Ariana Grande and Lady Gaga, blending catchy hooks with personal lyrics.

2. **The Blind Spots** - This indie band combines elements of rock and pop, echoing some Fleetwood Mac influences while bringing a fresh local twist.

3. **Taina Asili** - Known for her powerful vocals and eclectic style, her music incorporates pop and world sounds that could resonate with fans of BlackPink and Lady Gaga.

4. **The Lonely Hearts** - A R&B and soul band that delivers smooth melodies and harmonies, appealing to fans of Drake and The Weeknd.

5. **Chinchilla** - An up-and-coming artist who fuses electronic pop with contemporary sounds, making her music a potential favorite for those who enjoy Ariana Grande and The Weeknd.

These artists reflect a mix of the pop

In [10]:
prompt_1_gemma_system = "You are an expert in music recommendation. Your specialty is in recommending local artists based on a user's specified, preferred artists."
prompt_1_gemma = """You are presented with a client who frequently listens to the following artists:
Ariana Grande
Fleetwood Mac
BlackPink
Lady Gaga
Drake
The Weeknd

You are asked to use your expert knowledge of these artists and your specialiation in local music to recommend a set of 5 artists from Ithaca, New York.
Provide these recommendations in a concise list.
"""
gw_1 = GemmaWrapper(gemma_model, gemma_tokenizer, prompt_1_gemma_system)
text = gw_1.get_response(prompt_1_gemma)
print(text)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Okay, fantastic! Based on your listening habits – Ariana Grande’s pop polish, Fleetwood Mac’s layered harmonies and soulful vibes, Blackpink’s rhythmic precision, Lady Gaga’s theatrical flair, Drake’s R&B sensibilities, and The Weeknd’s atmospheric darkness – you clearly appreciate strong vocals, production quality, and a blend of pop, R&B, and a touch of alternative. 

I’ve focused on Ithaca, NY artists that share some of those core elements. Here are 5 recommendations, with a little explanation of why they might resonate:

1.  **The Wandering Eyes:** (Indie Folk/Pop) – They have a similar layered vocal approach to Fleetwood Mac, with beautiful harmonies and thoughtful songwriting. They create a dreamy, atmospheric soundscape. 

2.  **Luna Kat:** (Indie Pop/R&B) – Luna Kat’s music has a definite Ariana Grande-esque vocal delivery and a strong pop sensibility, but with a more indie, slightly experimental edge. 

3.  **Riverdale Church:** (Alternative/Psych-Pop) –  They lean into the da

In [ ]:
prompt_2 = """You are an expert in music recommendation. Your specialty is in ranking a list of artists by how similar each one is to a different set of artists that someone already knows.
You are presented with a client who frequently listens to the following artists:
{seeds}
You are asked to use your expert knowledge of these artists to rank the following artists (on which you are also an expert) in order from most recommended to least recommended:
{candidates}
You must present these recommendations in a very specific way. Each candidate artist that you are recommending has an integer ID associated with them. The key is as follows:
{candidate_key}
You must finish your response by listing your recommendations only using these ids, separated by commas, and surrounded by <>.
An example recommendation would be as follows:
<#,#,#,#,#,#,#,#,#,#,...>
With each hashtag replaced by the artist you recommend in that position. You must abide by the following rules:
- Rank all of the artists in the candidate set I provided you, not just however many are in my example recommendation
- Do not include any artists not in that candidate set
- Your response must be formatted as I have said, in a list of comma separated ids (governed by the key I provided) and surrounded by angle brackets/chevrons (<>)
"""

gpt_2 = GPTRecommender(artists, prompt_2, client)
evaluator_gpt_2 = Evaluator(gpt_2, artist_ids, seeds)
gpt_2_score = evaluator_gpt_2.eval_model()
print("Experiment 2 ChatGPT Score:", gpt_2_score)


Streaming output truncated to the last 5000 lines.
2. MGMT
3. Oliver Tree
4. Bastille
5. The Wrecks
6. The Fray
7. Jason Isbell and the 400 Unit
8. Hanan Ben Ari
9. Brent Faiyaz
10. Angel Olsen
11. Matt Maeson
12. Hillsong UNITED 
13. Hillsong Young & Free
14. Eyal Golan
15. Craig Xen
16. The HU
17. Zelooperz
18. Chico Buarque
19. Curtis Mayfield
20. David Guetta
21. Omer Adam
22. Stellar
23. Ratboys
24. Bobby Vinton
25. Title Fight
26. Kurt Cobain
27. Yello
28. Charmer
29. Ishay Ribo
30. Taurus Wells

Following the ranking, here are the IDs of the recommended artists:

<7,32,9,33,15,17,0,5,4,30,28,23,8,16,1,2,20,12,18,25,27,3,14,10,24,22,19,21,31,29>

Difference 4
Padding: [0, 0, 1, 0]
[1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0]
Score: 0.6423611111111112
['17175c38-aa04-4c34-adc4-b31fe0593677', '55f1a161-6c24-468f-9a88-495f9ab27b40', 'dbb2b8e5-1a21-4f0f-8b6f-41728593c7cf', 'ba260d4b-eb46-4baa-af4c-ce749749b455', '86f00fec-127e

In [ ]:
system_prompt_2 = "You are an expert in music recommendation. Your specialty is in ranking a list of artists by how similar each one is to a different set of artists that someone already knows."
prompt_2 = """
You are presented with a client who frequently listens to the following artists:
{seeds}
You are asked to use your expert knowledge of these artists to rank the following artists (on which you are also an expert) in order from most recommended to least recommended:
{candidates}
You must present these recommendations in a very specific way. Each candidate artist that you are recommending has an integer ID associated with them. The key is as follows:
{candidate_key}
You must finish your response by listing your recommendations only using these ids, separated by commas, and surrounded by <>.
An example recommendation would be as follows:
<#,#,#,#,#,#,#,#,#,#,...>
With each hashtag replaced by the artist you recommend in that position.
Provide me with only the ranked list of integer IDs associated with the ranking, do not enumerate your thought process.
"""
seeds_trunc = [x for x in seeds]
shuffle(seeds_trunc)
seeds_trunc = seeds_trunc[:30]
gw_2 = GemmaWrapper(gemma_model, gemma_tokenizer, system_prompt_2)
gemma_2 = InternalRecommender(artists, prompt_2, gw_2)
evaluator_gemma_2 = Evaluator(gemma_2, artist_ids, seeds_trunc)
gemma_2_score = evaluator_gemma_2.eval_model()
print("Experiment 2 Gemma Score:", gemma_2_score)

['2d49a351-1b71-4ff0-a661-62e127d924c4', 'f786d1cf-ee78-45c3-b413-5a173ae01ea4', '9c027db0-3e45-48cc-bc67-2b7bb278458d', '7b61102f-1870-499f-bccf-bc978e39a649', 'de585fd5-0bc6-4c6a-bd12-acb28b16d4f1', 'd4be6dd5-81cc-4592-be64-5d545e41fd96', '5f50d015-245d-452a-8306-9ac73c2b034c']
['e57fe676-301e-40ae-ad94-aa74489074e7', '9fb78b90-04c5-4981-9099-a0ed879db89d', '25177825-d30c-4759-be64-c130570617a1', '8e3e2f1f-a1b8-423d-a0af-fe5279b3ba21', '5818145d-52fb-4b40-aea7-7697efbd1a25', '7ecad6ef-e439-4bab-9d77-865d898735de', '8a8e491e-7a2e-4d33-8c70-8e6fd2c02fd4', '69dd91bf-6cf7-4e40-b05b-c1a622fb16dd', '1180e1df-4c58-4573-9f03-35d5984cfa03', '15debe80-cf8b-4c61-82f4-ee2d3ffb505f', '466286c4-4a95-4c20-89bb-6a902e6172a5', '7df28cc6-7887-4b64-b113-6fdec39fe930', 'ea596e03-92ab-44f7-bd5e-bd968a7e3c06', '58151a96-41fa-4e70-ac17-52a44530c947']
<11, 5, 13, 3, 2, 10, 4, 8, 9, 0, 1, 7, 12, 6, 1>

Response Text: 
<11, 5, 13, 3, 2, 10, 4, 8, 9, 0, 1, 7, 12, 6, 1>


Difference -1
[0, 1, 1, 0, 1, 1, 0, 0, 

This evaluation was altered and re-run on a separate computer, so this output is invalid. For the Experiment 2 Gemma Score, see the paper.

In [24]:
prompt_3_gpt = """You are an expert in music recommendation. Your specialty is in ranking a list of artists by how similar each one is to a different set of artists that someone already knows.
You are presented with a client who frequently listens to the following artists:
{seeds}

You are asked to use your expert knowledge of these artists to rank the following artists (on which you are also an expert) in order from most recommended to least recommended:
{candidates}

You must present these recommendations in a very specific way. Each candidate artist that you are recommending has an integer ID associated with them. The key is as follows:
{candidate_key}

You must finish your response by listing your recommendations only using these ids, separated by commas, and surrounded by <>.
An example recommendation would be as follows:
<#,#,#,#,#,#,#,#,#,#,...>
With each hashtag replaced by the artist you recommend in that position. You must abide by the following rules:
- Rank all of the artists in the candidate set I provided you, not just however many are in my example recommendation
- Do not include any artists not in that candidate set
- Your response must be formatted as I have said, in a list of comma separated ids (governed by the key I provided) and surrounded by angle brackets/chevrons (<>)
"""

gpt_3 = GPTRecommender(artists, prompt_3_gpt, client)
evaluator_gpt_3 = Evaluator(gpt_3, artist_ids, seeds)
gpt_3_score = evaluator_gpt_3.eval_model(use_genres=True)
print("Experiment 3 ChatGPT Score:", gpt_3_score)

Streaming output truncated to the last 5000 lines.
Here’s the ranked list of recommendations based on the similarity to the artists you're familiar with:

1. **SZA** (10) - Her blend of R&B and alternative elements resonates well with artists like The Weeknd and Kali Uchis.
2. **Doja Cat** (41) - A mix of pop and hip hop with R&B influences makes her a great match for your tastes.
3. **Tricky** (7) - Offers a unique blend of downtempo and electronic influences, similar to Gorillaz.
4. **Nelly Furtado** (2) - Her dance pop style and varied influences align well with many artists you enjoy.
5. **Megan Thee Stallion** (36) - Energetic hip hop and pop rap align with your preference for artists like Cardi B and Drake.
6. **Taylor Swift** (4) - Her versatility across pop, country, and alternative resonates with your eclectic taste.
7. **Pitbull** (8) - Dance pop and hip hop blend perfectly for fans of upbeat music like Flo Rida.
8. **Black Eyed Peas** (5) - Their fusion of hip hop and pop is

In [ ]:
system_prompt_3 = "You are an expert in music recommendation. Your specialty is in ranking a list of artists by how similar each one is to a different set of artists that someone already knows."
prompt_3 = """
You are presented with a client who frequently listens to the following artists:
{seeds}

You are asked to use your expert knowledge of these artists to rank the following artists (on which you are also an expert) in order from most recommended to least recommended:
{candidates}

You must present these recommendations in a very specific way. Each candidate artist that you are recommending has an integer ID associated with them. The key is as follows:
{candidate_key}

You must finish your response by listing your recommendations only using these ids, separated by commas, and surrounded by <>.
An example recommendation would be as follows:
<#,#,#,#,#,#,#,#,#,#,...>
With each hashtag replaced by the artist you recommend in that position.
Provide me with only the ranked list of integer IDs associated with the ranking, do not enumerate your thought process.
"""
gw_3 = GemmaWrapper(gemma_model, gemma_tokenizer, system_prompt_3)
gemma_3 = InternalRecommender(artists, prompt_3, gw_3)
evaluator_gemma_3 = Evaluator(gemma_3, artist_ids, seeds)
gemma_3_score = evaluator_gemma_3.eval_model(use_genres=True)
print("Experiment 2 Gemma Score:", gemma_3_score)

This cell was run on a separate computer, so there is no output here. For the results of this evaluation, see the paper.

In [40]:
prompt_4_gpt = """You are an expert in music recommendation. Your specialty is in ranking a list of artists by how similar each one is to a different set of artists that someone already knows.
You are presented with a client who frequently listens to the following artists (I have also provided the first 5 sentences of their wikipedia pages for context):
{seeds}

You are asked to use your expert knowledge of these artists to rank the following artists (on which you are also an expert) in order from most recommended to least recommended:
{candidates}

You must present these recommendations in a very specific way. Each candidate artist that you are recommending has an integer ID associated with them. The key is as follows:
{candidate_key}

You must finish your response by listing your recommendations only using these ids, separated by commas, and surrounded by <>.
An example recommendation would be as follows:
<#,#,#,#,#,#,#,#,#,#,...>
With each hashtag replaced by the artist you recommend in that position. You must abide by the following rules:
- Rank all of the artists in the candidate set I provided you, not just however many are in my example recommendation
- Do not include any artists not in that candidate set
- Your response must be formatted as I have said, in a list of comma separated ids (governed by the key I provided) and surrounded by angle brackets/chevrons (<>)
"""

gpt_4 = GPTRecommender(artists, prompt_4_gpt, client)
evaluator_gpt_4 = Evaluator(gpt_4, artist_ids, seeds)
gpt_4_score = evaluator_gpt_4.eval_model(use_wiki=True)
print("Experiment 4 ChatGPT Score:", gpt_4_score)

Streaming output truncated to the last 5000 lines.
15. **The Last Dinner Party** - Their indie rock style fits well within the recommendations of contemporary rock and pop.
16. **Slow Pulp** - Their indie rock vibe can appeal to fans of artists similar to Beach House and Whitne.
17. **Johnny Flynn** - His folk influences might resonate with audiences that enjoy Jim Croce and singer-songwriters.
18. **Buffalo Springfield** - Known for their folk-rock sound, they could appeal to fans of classic rock influences.
19. **Merle Haggard** - His classic country sound could resonate with fans of Americana and lyrical storytelling.
20. **Steely Dan** - Theirfusion of rock, jazz, and sophisticated lyrics would appeal to fans of music with depth.
21. **L'Impératrice** - Their blend of pop and disco might attract listeners who enjoy upbeat and eclectic sounds.
22. **Chromeo** - Their electro-funk sound resonates with electronic music fans looking to explore funky modern additions.
23. **G-Unit** - T